In [1]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid, MNISTSuperpixels
from torch_geometric.data import DataLoader


import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv


In [2]:
dataset = 'Mnist'
path = '../data/geometric/MNIST'
dataset = MNISTSuperpixels(path, dataset, T.Distance())
loader = DataLoader(dataset, batch_size=32, shuffle=True)


# dataset = Planetoid(path, dataset, T.NormalizeFeatures())
#data = dataset[0]

In [3]:
print(dataset.num_features)

1


In [28]:
# class Net(torch.nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = GCNConv(dataset.num_features, 16, improved=False)
#         self.conv2 = GCNConv(16, dataset.num_classes, improved=False)
#         # self.conv1 = ChebConv(data.num_features, 16, K=2)
#         # self.conv2 = ChebConv(16, data.num_features, K=2)
        
#     def forward(self):
#         x, edge_index = data.x, data.edge_index
#         x = F.relu(self.conv1(x, edge_index))
#         x = F.dropout(x, training=self.training)
#         x = self.conv2(x, edge_index)
#         return F.log_softmax(x, dim=1)


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, improved=False)
        self.conv2 = GCNConv(16, dataset.num_classes, improved=False)
        # self.conv1 = ChebConv(data.num_features, 16, K=2)
        # self.conv2 = ChebConv(16, data.num_features, K=2)
        
        self.deconv1 = GCNConv(dataset.num_classes, 16, improved=False)
        self.deconv2 = GCNConv(16, dataset.num_features, improved=False)
        
    def encode(self, x, edge_index):
        
        x = F.relu(self.conv1(x, edge_index))
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return x, edge_index
    
    def decode(self, x, edge_index):
        x = F.relu(self.deconv1(x, edge_index))
        #x = F.dropout(x, training=self.training)
        x = self.deconv2(x, edge_index)
        
        return x
        
        
    def forward(self):
        
        x, edge_index = data.x, data.edge_index
        
        encoded, idx = self.encode(x, edge_index)
        decoded = self.decode(encoded, idx)
        
        return decoded
    


In [ ]:
# Pyro Example: http://pyro.ai/examples/vae.html
# VAE example with sampling code 
# https://wiseodd.github.io/techblog/2017/01/24/vae-pytorch/
# https://github.com/wiseodd/generative-models/blob/master/VAE/vanilla_vae/vae_pytorch.py
    
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
        
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)


Wxh = xavier_init(size=[X_dim, h_dim])
bxh = Variable(torch.zeros(h_dim), requires_grad=True)

Whz_mu = xavier_init(size=[h_dim, Z_dim])
bhz_mu = Variable(torch.zeros(Z_dim), requires_grad=True)

Whz_var = xavier_init(size=[h_dim, Z_dim])
bhz_var = Variable(torch.zeros(Z_dim), requires_grad=True)


def Q(X):
    h = nn.relu(X @ Wxh + bxh.repeat(X.size(0), 1))
    z_mu = h @ Whz_mu + bhz_mu.repeat(h.size(0), 1)
    z_var = h @ Whz_var + bhz_var.repeat(h.size(0), 1)
    return z_mu, z_var

def sample_z(mu, log_var):
    # Using reparameterization trick to sample from a gaussian
    eps = Variable(torch.randn(mb_size, Z_dim))
    return mu + torch.exp(log_var / 2) * eps

Wzh = xavier_init(size=[Z_dim, h_dim])
bzh = Variable(torch.zeros(h_dim), requires_grad=True)

Whx = xavier_init(size=[h_dim, X_dim])
bhx = Variable(torch.zeros(X_dim), requires_grad=True)


def P(z):
    h = nn.relu(z @ Wzh + bzh.repeat(z.size(0), 1))
    X = nn.sigmoid(h @ Whx + bhx.repeat(h.size(0), 1))
    return X


solver = optim.Adam(params, lr=lr)

for it in range(100000):
    X, _ = mnist.train.next_batch(mb_size)
    X = Variable(torch.from_numpy(X))

    # Forward
    z_mu, z_var = Q(X)
    z = sample_z(z_mu, z_var)
    X_sample = P(z)

    # Loss
    recon_loss = nn.binary_cross_entropy(X_sample, X, size_average=False) / mb_size
    kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
    loss = recon_loss + kl_loss

    # Backward
    loss.backward()

    # Update
    solver.step()

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

numEpochs = 5

In [54]:
for epoch in range(1, numEpochs+1):
    for step, batch in enumerate(loader):
        if step < 20:
            
            data = batch.to(device)
            model.train()
            optimizer.zero_grad()

            loss = nn.MSELoss()
#             loss = nn.BCELoss()

            target = data
            output = loss(model(), target['x'])
            output.backward()


#           F.binary_cross_entropy(model(), target['x']).backward()
            optimizer.step()
        
            if step % 10 == 0:
                print('Epoch: ', epoch, 'step: ', step, 'train loss: ', output.item())
        else: 
            break
    

#             # plotting decoded image (second row)
#             _, decoded_data = autoencoder(view_data)
#             for i in range(N_TEST_IMG):
#                 a[1][i].clear()
#                 a[1][i].imshow(np.reshape(decoded_data.data.numpy()[i], (28, 28)), cmap='gray')
#                 a[1][i].set_xticks(()); a[1][i].set_yticks(())
#             plt.draw(); plt.pause(0.05)

Epoch:  1 step:  0 train loss:  0.17328667640686035
Epoch:  1 step:  10 train loss:  0.09969157725572586
Epoch:  2 step:  0 train loss:  0.0971202403306961
Epoch:  2 step:  10 train loss:  0.0872308686375618
Epoch:  3 step:  0 train loss:  0.0834479033946991
Epoch:  3 step:  10 train loss:  0.08151454478502274
Epoch:  4 step:  0 train loss:  0.0771080031991005
Epoch:  4 step:  10 train loss:  0.0720825120806694
Epoch:  5 step:  0 train loss:  0.0752023234963417
Epoch:  5 step:  10 train loss:  0.07494006305932999


In [7]:
# def train():
#     model.train()
#     optimizer.zero_grad()
    
#     F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    
#     loss = nn.BCELoss()
#     target = data.x[data.train_mask]
#     print(target)
#     print(model()[data.train_mask])
    
#     output = loss(model()[data.train_mask], target)
#     output.backward()
    
    
#     torch.nn.BCELoss(model()[data.train_mask], data.x[data.train_mask]).backward()
    
#     optimizer.step()

# def test():
#     model.eval()
#     logits, accs = model(), []
#     for _, mask in data('train_mask', 'val_mask', 'test_mask'):
#         pred = logits[mask].max(1)[1]
#         acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
#         accs.append(acc)
#     return accs

# best_val_acc = test_acc = 0
# for epoch in range(1, 50):
#     train()
#     train_acc, val_acc, tmp_test_acc = test()
#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         test_acc = tmp_test_acc
#     log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
#     print(log.format(epoch, train_acc, best_val_acc, test_acc))


In [37]:
print(min(target['x']))

tensor([0.])
